In [92]:
import torch, numpy as np, scipy.sparse as sp
import torch.optim as optim, torch.nn.functional as F
import torch_sparse
import pickle
import random
#from  model import HATT
from torch_scatter import scatter
from collections import defaultdict
import time
import torch.nn as nn, torch.nn.functional as F
import math
from torch_geometric.utils import softmax
import sklearn.metrics
import os

from scipy.sparse import lil_matrix
import argparse
from graph.graph import LaplaceGraph
from codes.dataset import FeaturesData, BPRTrainLoader, UserItemData
from codes.performance import evaluate
from session.run import Session
from torch.utils.data import DataLoader

In [33]:
import os
import torch
import numpy as np
import scipy.sparse as sp 
from torch.utils.data import Dataset

In [35]:
device = torch.device('cuda:0')

In [34]:
def sparse_ones(indices, size, dtype=torch.float):
    one = torch.ones(indices.shape[1], dtype=dtype)
    return torch.sparse.FloatTensor(indices, one, size=size).to(dtype)

def to_tensor(graph):
    graph = graph.tocoo()
    values = graph.data
    indices = np.vstack((graph.row, graph.col))
    graph = torch.sparse.FloatTensor(torch.LongTensor(indices), torch.FloatTensor(values), 
                                          torch.Size(graph.shape))
    return graph

In [38]:
with open('1101_1108/c_u_train.txt','r') as file:
    length = 0
    culist = []
    while 1:
        line = file.readline()
        if(line==''):
            break
        n = len(line)
        s = line[0:n-1].split(',')
        #print(s)
        length += (len(s)-1)
        for j in range(len(s)-1):
            #print(s[0],s[j+1])
            culist.append([int(s[0]),int(s[j+1])])

In [40]:
with open('1101_1108/c_u_test.txt','r') as file:
    length = 0
    culist_test = []
    while 1:
        line = file.readline()
        if(line==''):
            break
        n = len(line)
        s = line[0:n-1].split(',')
        #print(s)
        length += (len(s)-1)
        for j in range(len(s)-1):
            #print(s[0],s[j+1])
            culist_test.append([int(s[0]),int(s[j+1])])

In [46]:
with open('1101_1108/u_a.txt','r') as file:
    length = 0
    ualist = []
    while 1:
        line = file.readline()
        if(line==''):
            break
        n = len(line)
        s = line[0:n-1].split(',')
        #print(s)
        length += (len(s)-1)
        for j in range(len(s)-1):
            #print(s[0],s[j+1])
            ualist.append([int(s[0]),int(s[j+1])])

In [58]:
with open('1101_1108/com_user_train.txt', 'w') as file:
    for i in range(len(culist)):
        file.write(str(culist[i][0]))
        file.write(',')
        file.write(str(culist[i][1]))
        file.write('\n')

In [59]:
with open('1101_1108/com_user_test.txt', 'w') as file:
    for i in range(len(culist_test)):
        file.write(str(culist_test[i][0]))
        file.write(',')
        file.write(str(culist_test[i][1]))
        file.write('\n')

In [62]:
with open('1101_1108/user_attr.txt', 'w') as file:
    for i in range(len(ualist)):
        file.write(str(ualist[i][0]))
        file.write(',')
        file.write(str(ualist[i][1]))
        file.write('\n')

In [71]:
n_users_test =  671
n_users_train = 7771
c = n_users_train+n_users_test
n_coms = 453
n_attrs = 94

In [74]:
with open('1101_1108/com_user_train.txt', 'r') as f:
    C_U_train_pairs = list(map(lambda s: tuple(int(i) for i in s[:-1].split(',')), f.readlines()))
indice = np.array(C_U_train_pairs, dtype=np.int32)
values = np.ones(len(C_U_train_pairs), dtype=np.float32)
ground_truth_c_u_train = sp.coo_matrix(
    (values, (indice[:, 0], indice[:, 1])), shape=(n_coms, n_users)).tocsr()

with open('1101_1108/com_user_test.txt', 'r') as f:
    C_U_test_pairs = list(map(lambda s: tuple(int(i) for i in s[:-1].split(',')), f.readlines()))
indice = np.array(C_U_test_pairs, dtype=np.int32)
values = np.ones(len(C_U_test_pairs), dtype=np.float32)
ground_truth_c_u_test = sp.coo_matrix(
    (values, (indice[:, 0], indice[:, 1])), shape=(n_coms, n_users)).tocsr()

with open('1101_1108/user_attr.txt', 'r') as f:
    U_A_pairs = list(map(lambda s: tuple(int(i) for i in s[:-1].split(',')), f.readlines()))
indice = np.array(U_A_pairs, dtype=np.int32)
values = np.ones(len(U_A_pairs), dtype=np.float32)
ground_truth_u_a = sp.coo_matrix(
    (values, (indice[:, 0], indice[:, 1])), shape=(n_users, n_attrs)).tocsr()

In [78]:
raw_graph = [ground_truth_c_u_train, ground_truth_u_a]

In [79]:
def laplace_transform(graph):
    rowsum_sqrt = sp.diags(1/(np.sqrt(graph.sum(axis=1).A.ravel()) + 1e-8))
    colsum_sqrt = sp.diags(1/(np.sqrt(graph.sum(axis=0).A.ravel()) + 1e-8))
    graph = rowsum_sqrt @ graph @ colsum_sqrt
    return graph

def to_tensor(graph):
    graph = graph.tocoo()
    values = graph.data
    indices = np.vstack((graph.row, graph.col))
    graph = torch.sparse.FloatTensor(torch.LongTensor(indices), torch.FloatTensor(values), 
                                          torch.Size(graph.shape))
    return graph

In [80]:
ground_truth_c_u_train.shape

(453, 8442)

In [83]:
ground_truth_u_a.shape

(8442, 94)

In [88]:
c_a_graph = lil_matrix((n_coms, n_attrs))

In [89]:
c_a_graph

<453x94 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in List of Lists format>

In [81]:
UC_graph = sp.bmat([[sp.identity(raw_graph[0].shape[1])-sp.identity(raw_graph[0].shape[1]), raw_graph[0].T],
                    [raw_graph[0], sp.identity(raw_graph[0].shape[0])-sp.identity(raw_graph[0].shape[0])]])

In [84]:
UA_graph = sp.bmat([[sp.identity(raw_graph[1].shape[0])-sp.identity(raw_graph[1].shape[0]), raw_graph[1]],
                    [raw_graph[1].T, sp.identity(raw_graph[1].shape[1])-sp.identity(raw_graph[1].shape[1])]])

In [90]:
#U-C-A
whole_graph = sp.bmat([[sp.identity(raw_graph[1].shape[0])-sp.identity(raw_graph[1].shape[0]), raw_graph[0].T, raw_graph[1]],
                     [raw_graph[0], sp.identity(raw_graph[0].shape[0])-sp.identity(raw_graph[0].shape[0]),c_a_graph],
                     [raw_graph[1].T, c_a_graph.T, sp.identity(raw_graph[1].shape[1])-sp.identity(raw_graph[1].shape[1])]])

In [91]:
whole_graph

<8989x8989 sparse matrix of type '<class 'numpy.float64'>'
	with 473950 stored elements in COOrdinate format>

In [30]:
class LightGCN(nn.Module):

    def __init__(self, n_users, n_coms, n_attrs, raw_graph, args, device):
        
        super(LightGCN, self).__init__()
        
        self.n_users = n_users
        self.n_coms = n_coms
        self.n_attrs = n_attrs
        self.emb_size = args.emb_size
        self.batch_size = args.batch_size
        self.decay = args.decay
        self.layers = args.layers
        self.device = device
        self.cu_graph, self.ua_graph = raw_graph
        self.mess_dropout = nn.Dropout(args.drop_rate, True)
        self.node_dropout = nn.Dropout(args.drop_rate, True)
        

        user_emb_weight = self._weight_init(user_feat, n_users, args.emb_size)
        com_emb_weight = self._weight_init(com_feat, n_coms, args.emb_size)
        attr_emb_weight = self._weight_init(attr_feat, n_attrs, args.emb_size)

        self.user_embeddings = nn.Embedding(self.n_users, self.emb_size, _weight=user_emb_weight)
        self.com_embeddings = nn.Embedding(self.n_coms, self.emb_size, _weight=com_emb_weight)
        self.attr_embeddings = nn.Embedding(self.n_attrs, self.emb_size, _weight=attr_emb_weight)
        
        
        c_a_graph = lil_matrix((n_coms, n_attrs))
        UC_graph = sp.bmat([[sp.identity(raw_graph[0].shape[1])-sp.identity(raw_graph[0].shape[1]), raw_graph[0].T],
                    [raw_graph[0], sp.identity(raw_graph[0].shape[0])-sp.identity(raw_graph[0].shape[0])]])
        self.uc_graph_tensor = to_tensor(laplace_transform(UC_graph)).to(device)
        
        UA_graph = sp.bmat([[sp.identity(raw_graph[1].shape[0])-sp.identity(raw_graph[1].shape[0]), raw_graph[1]],
                    [raw_graph[1].T, sp.identity(raw_graph[1].shape[1])-sp.identity(raw_graph[1].shape[1])]])
        self.ua_graph_tensor = to_tensor(laplace_transform(UA_graph)).to(device)
        
        whole_graph = sp.bmat([[sp.identity(raw_graph[1].shape[0])-sp.identity(raw_graph[1].shape[0]), raw_graph[0].T, raw_graph[1]],
                     [raw_graph[0], sp.identity(raw_graph[0].shape[0])-sp.identity(raw_graph[0].shape[0]),c_a_graph],
                     [raw_graph[1].T, c_a_graph.T, sp.identity(raw_graph[1].shape[1])-sp.identity(raw_graph[1].shape[1])]])
        self.whole_graph_tensor = to_tensor(laplace_transform(whole_graph)).to(device)

    def _weight_init(self, feat, rows, cols):

        if feat is None:
            free_emb = nn.init.normal_(torch.empty(rows, cols), std=0.01)
            return free_emb
        else:
            free_emb = nn.init.normal_(torch.empty(rows, cols - feat.shape[-1]), std=0.01)
            feat_emb = torch.tensor(feat) * 0.01
            return torch.cat([free_emb, feat_emb], dim=1)
        
    def forward(self, graph):
        # node dropout on graph
        indices = graph._indices()
        values = graph._values()
        values = self.node_dropout(values)
        graph = torch.sparse.FloatTensor(
            indices, values, size=graph.shape)
        # propagate
        features = torch.cat([self.user_embeddings.weight, self.com_embeddings.weight, self.attr_embeddings.weight], dim=0)
        all_features = [features]
        for i in range(self.num_layers):
            features = self.mess_dropout(torch.matmul(graph, features))
            all_features.append(F.normalize(features))
            
        temp_features = all_features

        all_features = torch.stack(all_features, dim = 1)
        all_features = torch.mean(all_features, dim = 1)
        A_feature, B_feature, C_feature= torch.split(
            all_features, (n_users, n_coms, n_attrs), 0)
        return A_feature, B_feature, C_feature, temp_features
    
    def evaluate(self, propagate_result, groups):
        users_feature, items_feature, groups_feature,_ = propagate_result
        groups_feature_atom, groups_feature_non_atom = [i[groups] for i in groups_feature] 
        items_feature_atom, items_feature_non_atom = items_feature 
        scores = torch.mm(groups_feature_atom, items_feature_atom.t()) 
        return scores

    def predict(self, items_feature, groups_feature):
        items_feature_atom, items_feature_non_atom = items_feature 
        groups_feature_atom, groups_feature_non_atom = groups_feature
        pred = torch.sum(items_feature_atom * groups_feature_atom, 2)#+ torch.sum(items_feature_non_atom * groups_feature_non_atom, 2)
        return pred

    def get_embedding(self, user_emb, item_emb, users, pos_items, neg_items):
        u_emb = user_emb[users]
        pos_emb = item_emb[pos_items]
        neg_emb = item_emb[neg_items]

        return u_emb, pos_emb, neg_emb
    

    def bpr_loss(self, user_emb, pos_emb, neg_emb):
        
        pos_score = torch.sum(user_emb * pos_emb, dim=1)
        neg_score = torch.sum(user_emb * neg_emb, dim=1)
        mf_loss = torch.mean(F.softplus(neg_score - pos_score))
        
        reg_loss = (1/2) * (user_emb.norm(2).pow(2) + 
                            pos_emb.norm(2).pow(2) + 
                            neg_emb.norm(2).pow(2)) / user_emb.shape[0] * self.decay

        loss = mf_loss + reg_loss
        return loss, mf_loss, reg_loss

In [31]:
root = '/notebook/HATT/lightgcn'

import sys
sys.path.append(root)

In [32]:
def parse_args():
    parser = argparse.ArgumentParser(description="gcn")
    parser.add_argument('--dataset_name', default='gowalla', type=str)
    parser.add_argument('--data_path', default=root + '/data', type=str)
    
    parser.add_argument('--emb_size', default=64, type=int)
    parser.add_argument('--num_epoch', default=1000, type=int)
    parser.add_argument('--lr', default=0.001, type=float)
    parser.add_argument('--decay', default=0.001, type=float)
    parser.add_argument('--layers', default=3, type=int)
    parser.add_argument('--batch_size', default=4096, type=int)

    parser.add_argument('--topks', default='[10,20]', type=str)
    parser.add_argument('--log', default=root + '/log/lightgcn.txt', type=str)
    parser.add_argument('--parameters_path', default= root + '/parameters/lightgcn', type=str)
    parser.add_argument('--cores', default=4, type=int)
    
    return parser.parse_args(args=[])


def get_dataloader(train_set, train_U2I, n_items, batch_size, cores):
    gcn_dataloader = BPRTrainLoader(train_set, train_U2I, n_items)
    gcn_dataloader = DataLoader(gcn_dataloader, batch_size, num_workers=cores, shuffle=True)

    return gcn_dataloader

def test(model, n_users, n_items):

    user_emb, item_emb = model.propagate()
    return user_emb.cpu().detach().numpy(), item_emb.cpu().detach().numpy()


if __name__ == '__main__':
    args = parse_args()

    # load movielens-1m
    # data = FeaturesData(args.data_path, args.dataset_name)
    # train_set, train_U2I, test_U2I, n_users, n_items, user_feat = data.load()

    # load gowall
    data = UserItemData(args.data_path, args.dataset_name)
    train_set, train_U2I, test_U2I, n_users, n_items = data.load()

    loader = get_dataloader(train_set, train_U2I, n_items, args.batch_size, args.cores)
    g = LaplaceGraph(n_users, n_items, train_U2I)
    adj = g.generate().cuda()

    gcn = LightGCN(n_users, n_items, adj, args)
    gcn = gcn.cuda()

    optimizer = optim.Adam(gcn.parameters(), lr=args.lr)

    sess = Session(gcn)

    f = open(args.log, 'w+')
    for epoch in range(args.num_epoch):

        loss = sess.train(loader, optimizer, args)
        print("epoch:{:d}, loss:[{:.6f}] = mf:[{:.6f}] + reg:[{:.6f}]".format(epoch+1, *loss))
        print("epoch:{:d}, loss:[{:.6f}] = mf:[{:.6f}] + reg:[{:.6f}]".format(epoch+1, *loss), file=f)

        gcn.eval()
        with torch.no_grad():
            user_emb, item_emb = test(gcn, n_users, n_items)
            perf_info = evaluate(user_emb,
                                 item_emb,
                                 n_users,
                                 n_items,
                                 train_U2I,
                                 test_U2I,
                                 args)

            print("recall@10:[{:.6f}], ndcg@10:[{:.6f}], recall@20:[{:.6f}], ndcg@20:[{:.6f}]".format(*perf_info), file=f)
            print("recall@10:[{:.6f}], ndcg@10:[{:.6f}], recall@20:[{:.6f}], ndcg@20:[{:.6f}]".format(*perf_info))
            
            # save embedding
            # torch.save((user_emb, item_emb), f=args.parameters_path + '_' + str(epoch + 1) + '.pth')
    f.close()

(spent time: 7.0741026401519775s)
epoch:1, loss:[0.528911] = mf:[0.527980] + reg:[0.000930]
(spent time: 7.881688356399536s)
recall@10:[0.051943], ndcg@10:[0.051303], recall@20:[0.075073], ndcg@20:[0.061611]
(spent time: 7.189534425735474s)
epoch:2, loss:[0.278241] = mf:[0.274569] + reg:[0.003672]
(spent time: 5.373518943786621s)
recall@10:[0.059028], ndcg@10:[0.058166], recall@20:[0.084658], ndcg@20:[0.069453]
(spent time: 7.08504843711853s)
epoch:3, loss:[0.214460] = mf:[0.209247] + reg:[0.005214]
(spent time: 5.470014333724976s)
recall@10:[0.063046], ndcg@10:[0.062957], recall@20:[0.091425], ndcg@20:[0.075436]
(spent time: 7.141435384750366s)
epoch:4, loss:[0.184809] = mf:[0.178683] + reg:[0.006126]
(spent time: 7.935750484466553s)
recall@10:[0.067485], ndcg@10:[0.067641], recall@20:[0.095714], ndcg@20:[0.080099]
(spent time: 7.083908319473267s)
epoch:5, loss:[0.163774] = mf:[0.157001] + reg:[0.006772]
(spent time: 5.442552804946899s)
recall@10:[0.069755], ndcg@10:[0.070747], recall

Process ForkPoolWorker-119:
Process ForkPoolWorker-118:
Process ForkPoolWorker-120:
Process ForkPoolWorker-117:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/l

KeyboardInterrupt: 